In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [10]:
df=pd.read_csv('HistoricalQuotes.csv')
# Convert Date to datetime format
df['Date'] = pd.to_datetime(df['Date'])
df.columns = df.columns.str.strip()

# Remove dollar signs and convert columns to float
for column in ['Close/Last', 'Open', 'High', 'Low']:
    df[column] = df[column].str.replace('$', '').astype(float)

# Sort by date
df = df.sort_values('Date')

# Normalize the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['Close/Last']])

# Create sequences
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
    return np.array(sequences)

seq_length = 3
sequences = create_sequences(scaled_data, seq_length)

X = sequences[:, :-1]
y = sequences[:, -1]

X = X.reshape((X.shape[0], X.shape[1], 1))

# Split into training and testing sets
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [11]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(seq_length - 1, 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

In [12]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50
63/63 [==============================] - 7s 23ms/step - loss: 0.0159 - val_loss: 0.0301
Epoch 2/50
63/63 [==============================] - 1s 8ms/step - loss: 0.0014 - val_loss: 0.0063
Epoch 3/50
63/63 [==============================] - 0s 8ms/step - loss: 5.7763e-04 - val_loss: 0.0041
Epoch 4/50
63/63 [==============================] - 1s 8ms/step - loss: 5.1404e-04 - val_loss: 0.0027
Epoch 5/50
63/63 [==============================] - 0s 8ms/step - loss: 4.7022e-04 - val_loss: 0.0015
Epoch 6/50
63/63 [==============================] - 1s 9ms/step - loss: 4.6398e-04 - val_loss: 6.3697e-04
Epoch 7/50
63/63 [==============================] - 1s 16ms/step - loss: 3.8912e-04 - val_loss: 9.5545e-04
Epoch 8/50
63/63 [==============================] - 0s 8ms/step - loss: 3.8633e-04 - val_loss: 3.7475e-04
Epoch 9/50
63/63 [==============================] - 1s 8ms/step - loss: 3.7277e-04 - val_loss: 2.6389e-04
Epoch 10/50
63/63 [==============================] - 1s 8ms/step - loss:

In [13]:
# Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

# Compare with actual values
actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# Print the predictions and actual values
for i in range(len(predictions)):
    print(f"Predicted: {predictions[i][0]}, Actual: {actual[i][0]}")


16/16 [==============================] - 1s 3ms/step
Predicted: 177.0247344970703, Actual: 178.12
Predicted: 176.55039978027344, Actual: 175.0
Predicted: 175.42169189453125, Actual: 176.21000000000004
Predicted: 173.67608642578125, Actual: 176.81999999999996
Predicted: 174.68051147460938, Actual: 176.67
Predicted: 175.0480499267578, Actual: 175.03
Predicted: 174.44911193847656, Actual: 176.94
Predicted: 173.9178466796875, Actual: 179.98
Predicted: 176.13302612304688, Actual: 181.72
Predicted: 178.7172088623047, Actual: 179.97
Predicted: 179.36355590820312, Actual: 178.44
Predicted: 177.71641540527344, Actual: 178.65
Predicted: 176.7449188232422, Actual: 178.02
Predicted: 176.69602966308594, Actual: 175.3
Predicted: 175.44496154785156, Actual: 175.24
Predicted: 173.5850067138672, Actual: 171.27
Predicted: 172.33990478515625, Actual: 168.85
Predicted: 168.91433715820312, Actual: 164.94
Predicted: 166.0851593017578, Actual: 172.77
Predicted: 165.8104248046875, Actual: 168.34
Predicted: 16

In [19]:
mse = mean_squared_error(actual, predictions)
mae = mean_absolute_error(actual, predictions)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")

Mean Squared Error: 34.16704062094532
Mean Absolute Error: 4.6286453702103785
